In [1]:
import pyspark
from importlib import reload

from pyspark.sql import SparkSession
import os
import pandas as pd
import sys
sys.path.insert(0, '/home/jovyan/work')
from script.helper.db_conn import db_connection
from datetime import datetime
from datetime import timezone

from script.helper.conn_prop import connection_properties
from script.helper.init_spark import initiate_spark
from script.etl.extract import *
from script.etl.load import *
import pandas as pd
from pyspark.sql.functions import to_timestamp
from pyspark.sql.functions import to_date

In [2]:

people=extract_csv('../data/people.csv')

===== Start Extracting ../data/people.csv data =====
===== Success Extracting ../data/people.csv data =====


In [3]:
relationships=extract_csv('../data/relationship.csv')

===== Start Extracting ../data/relationship.csv data =====
====== Failed to Extract Data ../data/relationship.csv ======,
 [PATH_NOT_FOUND] Path does not exist: file:/home/jovyan/work/data/relationship.csv.


In [4]:
relationships=extract_csv('../data/relationships.csv')

===== Start Extracting ../data/relationships.csv data =====
===== Success Extracting ../data/relationships.csv data =====


In [5]:
table_names=["acquisition","company","funds","investments","ipos","funding_rounds"]
for name in table_names:
    globals()[f'{name}']=extract_db_src(name)

===== Start Extracting acquisition data =====
===== Success Extracting acquisition data =====
===== Start Extracting company data =====
===== Success Extracting company data =====
===== Start Extracting funds data =====
===== Success Extracting funds data =====
===== Start Extracting investments data =====
===== Success Extracting investments data =====
===== Start Extracting ipos data =====
===== Success Extracting ipos data =====
===== Start Extracting funding_rounds data =====
===== Success Extracting funding_rounds data =====


In [6]:
relationships.show()

+---------------+----------------+----------------------+--------------------+--------------------+-------+--------+--------------------+--------------------+--------------------+
|relationship_id|person_object_id|relationship_object_id|            start_at|              end_at|is_past|sequence|               title|          created_at|          updated_at|
+---------------+----------------+----------------------+--------------------+--------------------+-------+--------+--------------------+--------------------+--------------------+
|              1|             p:2|                   c:1|                NULL|                NULL|  false|       8|Co-Founder/CEO/Bo...|2007-05-25 07:03:...|2013-06-03 09:58:...|
|              2|             p:3|                   c:1|                NULL|                NULL|  false|  279242|        VP Marketing|2007-05-25 07:04:...|2010-05-21 16:31:...|
|              3|             p:4|                   c:3|                NULL|                NULL| 

In [7]:
people.show()

+---------+---------+----------+----------+--------------------+--------------------+
|people_id|object_id|first_name| last_name|          birthplace|    affiliation_name|
+---------+---------+----------+----------+--------------------+--------------------+
|        1|      p:2|       Ben|   Elowitz|                NULL|           Blue Nile|
|        2|      p:3|     Kevin|  Flaherty|                NULL|            Wetpaint|
|        3|      p:4|      Raju|   Vegesna|                NULL|                Zoho|
|        4|      p:5|       Ian|     Wenig|                NULL|                Zoho|
|        5|      p:6|     Kevin|      Rose|         Redding, CA|        i/o Ventures|
|        6|      p:7|       Jay|   Adelson|         Detroit, MI|                Digg|
|        7|      p:8|      Owen|     Byrne|                NULL|                Digg|
|        8|      p:9|       Ron|Gorodetzky|                NULL|                Digg|
|        9|     p:10|      Mark|Zuckerberg|           

In [8]:
step="Data Staging"
process="Load"



In [9]:
load_stg(people,"people")

===== Get only new data =====
===== Start Extracting people data =====
===== Success Extracting people data =====
===== Already got only New Data  =====
===== Start Loading people new data =====
===== Success Loading people new data =====


In [10]:
people.dtypes

[('people_id', 'string'),
 ('object_id', 'string'),
 ('first_name', 'string'),
 ('last_name', 'string'),
 ('birthplace', 'string'),
 ('affiliation_name', 'string')]

In [11]:
relationships.dtypes

[('relationship_id', 'string'),
 ('person_object_id', 'string'),
 ('relationship_object_id', 'string'),
 ('start_at', 'string'),
 ('end_at', 'string'),
 ('is_past', 'string'),
 ('sequence', 'string'),
 ('title', 'string'),
 ('created_at', 'string'),
 ('updated_at', 'string')]

In [12]:


relationships=relationships.withColumn("start_at",to_date( relationships['start_at'], "yyyy-mm-dd")) 
relationships=relationships.withColumn("end_at",to_date( relationships['end_at'], "yyyy-mm-dd")) 
relationships=relationships.withColumn("created_at",to_timestamp( relationships['created_at'], "yyyy-mm-dd HH:mm:ss")) 
relationships=relationships.withColumn("updated_at",to_timestamp( relationships['updated_at'], "yyyy-mm-dd HH:mm:ss")) 

In [13]:
relationships.show()

+---------------+----------------+----------------------+----------+----------+-------+--------+--------------------+-------------------+-------------------+
|relationship_id|person_object_id|relationship_object_id|  start_at|    end_at|is_past|sequence|               title|         created_at|         updated_at|
+---------------+----------------+----------------------+----------+----------+-------+--------+--------------------+-------------------+-------------------+
|              1|             p:2|                   c:1|      NULL|      NULL|  false|       8|Co-Founder/CEO/Bo...|2007-01-25 07:03:54|2013-01-03 09:58:46|
|              2|             p:3|                   c:1|      NULL|      NULL|  false|  279242|        VP Marketing|2007-01-25 07:04:16|2010-01-21 16:31:34|
|              3|             p:4|                   c:3|      NULL|      NULL|  false|       4|          Evangelist|2007-01-25 19:33:03|2013-01-29 13:36:58|
|              4|             p:5|                  

In [14]:
relationships.dtypes

[('relationship_id', 'string'),
 ('person_object_id', 'string'),
 ('relationship_object_id', 'string'),
 ('start_at', 'date'),
 ('end_at', 'date'),
 ('is_past', 'string'),
 ('sequence', 'string'),
 ('title', 'string'),
 ('created_at', 'timestamp'),
 ('updated_at', 'timestamp')]

In [15]:
load_stg(relationships,"relationships")

===== Get only new data =====
===== Start Extracting relationships data =====
===== Success Extracting relationships data =====
===== Already got only New Data  =====
===== Start Loading relationships new data =====
===== Success Loading relationships new data =====


In [16]:
table_names=["acquisition","company","funds","investments","ipos","funding_rounds"]
for name in table_names:
    load_stg(globals()[f'{name}'],name)

===== Get only new data =====
===== Start Extracting acquisition data =====
===== Success Extracting acquisition data =====
===== Already got only New Data  =====
===== Start Loading acquisition new data =====
===== Success Loading acquisition new data =====
===== Get only new data =====
===== Start Extracting company data =====
===== Success Extracting company data =====
===== Already got only New Data  =====
===== Start Loading company new data =====
===== Success Loading company new data =====
===== Get only new data =====
===== Start Extracting funds data =====
===== Success Extracting funds data =====
===== Already got only New Data  =====
===== Start Loading funds new data =====
===== Success Loading funds new data =====
===== Get only new data =====
===== Start Extracting investments data =====
===== Success Extracting investments data =====
===== Already got only New Data  =====
===== Start Loading investments new data =====
===== Success Loading investments new data =====
====

# DATA STAGING ETL

## EXTRACT

In [17]:
table_names=["people","relationships","acquisition","company","funds","investments","ipos","funding_rounds"]
for name in table_names:
    globals()[f'stg_{name}']=extract_db_stg(name)

===== Start Extracting people data =====
===== Success Extracting people data =====
===== Start Extracting relationships data =====
===== Success Extracting relationships data =====
===== Start Extracting acquisition data =====
===== Success Extracting acquisition data =====
===== Start Extracting company data =====
===== Success Extracting company data =====
===== Start Extracting funds data =====
===== Success Extracting funds data =====
===== Start Extracting investments data =====
===== Success Extracting investments data =====
===== Start Extracting ipos data =====
===== Success Extracting ipos data =====
===== Start Extracting funding_rounds data =====
===== Success Extracting funding_rounds data =====


In [18]:
dim_term_code=stg_acquisition.select(stg_acquisition['term_code']).distinct()


In [19]:
dim_term_code.show()

+--------------+
|     term_code|
+--------------+
|         stock|
|cash_and_stock|
|          cash|
|              |
+--------------+



In [20]:
load_dwh(dim_term_code,"dim_term_code")

====== Failed to Load Data dim_term_code ====== 
 module 'pyspark' has no attribute 'pandas'


In [25]:
from pyspark import *

In [27]:
_,_,cp_stg,_=connection_properties()
_,_,stg_url,_= db_connection()

# Truncate the target table (ensure data integrity by removing old records before loading new data)
truncate_sql = f"TRUNCATE TABLE {dim_term_code} CASCADE"
pd.read_sql(truncate_sql,stg_url)

ArgumentError: Could not parse SQLAlchemy URL from string 'jdbc:postgresql://target_db_container:5432/warehouse'

In [41]:
_,_,cp_stg,_=connection_properties()
_,_,stg_url,_= db_connection()
table_name="dim_term_code"
truncate_sql = f"TRUNCATE TABLE {table_name} CASCADE"

# Use the jdbc connection to execute the SQL truncate statement
truncate_df = spark.read.format("jdbc").option("url", stg_url).option("query", "TRUNCATE TABLE dim_term_code CASCADE") \
            .option("user", "postgres") \
            .option("password", "cobapassword") \
            .load()

log_success(f"===== Truncated table {table_name} successfully =====")      

print(f"===== Start Loading {table_name} new data =====")


Py4JJavaError: An error occurred while calling o1183.load.
: org.postgresql.util.PSQLException: ERROR: syntax error at or near "TABLE"
  Position: 25
	at org.postgresql.core.v3.QueryExecutorImpl.receiveErrorResponse(QueryExecutorImpl.java:2713)
	at org.postgresql.core.v3.QueryExecutorImpl.processResults(QueryExecutorImpl.java:2401)
	at org.postgresql.core.v3.QueryExecutorImpl.execute(QueryExecutorImpl.java:368)
	at org.postgresql.jdbc.PgStatement.executeInternal(PgStatement.java:498)
	at org.postgresql.jdbc.PgStatement.execute(PgStatement.java:415)
	at org.postgresql.jdbc.PgPreparedStatement.executeWithFlags(PgPreparedStatement.java:190)
	at org.postgresql.jdbc.PgPreparedStatement.executeQuery(PgPreparedStatement.java:134)
	at org.apache.spark.sql.execution.datasources.jdbc.JDBCRDD$.getQueryOutputSchema(JDBCRDD.scala:68)
	at org.apache.spark.sql.execution.datasources.jdbc.JDBCRDD$.resolveTable(JDBCRDD.scala:58)
	at org.apache.spark.sql.execution.datasources.jdbc.JDBCRelation$.getSchema

In [ ]:


new_data.write.jdbc(url=stg_url,table=table_name,mode="append",properties=cp_stg)
process="Load"
log_success(step,process,source,table_name)
print(f"===== Success Loading {table_name} new data =====")

In [21]:
for name in table_names:
    print(globals()[f'stg_{name}'].dtypes, '\n')

[('people_id', 'string'), ('object_id', 'string'), ('first_name', 'string'), ('last_name', 'string'), ('birthplace', 'string'), ('affiliation_name', 'string')] 

[('relationship_id', 'string'), ('person_object_id', 'string'), ('relationship_object_id', 'string'), ('start_at', 'date'), ('end_at', 'date'), ('is_past', 'string'), ('sequence', 'string'), ('title', 'string'), ('created_at', 'timestamp'), ('updated_at', 'timestamp')] 

[('acquisition_id', 'int'), ('acquiring_object_id', 'string'), ('acquired_object_id', 'string'), ('term_code', 'string'), ('price_amount', 'decimal(15,2)'), ('price_currency_code', 'string'), ('acquired_at', 'timestamp'), ('source_url', 'string'), ('source_description', 'string'), ('created_at', 'timestamp'), ('updated_at', 'timestamp')] 

[('office_id', 'int'), ('object_id', 'string'), ('description', 'string'), ('region', 'string'), ('address1', 'string'), ('address2', 'string'), ('city', 'string'), ('zip_code', 'string'), ('state_code', 'string'), ('country

In [22]:
customer_birtday_transaction=tx_date.join(bd_date, 'customer_id',"right").select(tx_date.customer_id,tx_date.EarliestTransactionDate, bd_date.birth_date)

NameError: name 'tx_date' is not defined